In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import ast
%matplotlib inline


import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
from keras.models import Model, Sequential


from keras.layers import Dense, Activation, Convolution1D,Conv1D, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Dropout
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
data = pd.read_csv(os.path.join("datasets","ptb-xl-a-comprehensive-electrocardiographic-feature-dataset-1.0.1","ptb-xl-a-comprehensive-electrocardiographic-feature-dataset-1.0.1","features","12sl_features.csv"))

In [3]:
data.head()

,P_Area_I,P_PeakTime_I,Q_Area_I,Q_PeakTime_I,R_Area_I,R_PeakTime_I,S_Area_I,S_PeakTime_I,QRS_Balance_I,T_Area_I,...,T+_Dur_aVF,T-_Dur_aVF,T+_Amp_aVF,T-_Amp_aVF,T_Morph_aVF,T_DurFull_aVF,P_Dur_Global,P_Found_Global,HR__Global,P_Term_V1
0,0.264,64.0,0.000,0.0,0.737,30.0,0.000,0.0,629.0,0.819,...,208.0,0.0,0.151,0.000,1,208.0,112.0,1,64.0,0.000
1,-0.047,6.0,0.015,8.0,0.794,36.0,1.246,76.0,205.0,2.206,...,0.0,127.0,0.000,-0.078,-1,127.0,66.0,1,75.0,0.000
2,0.000,0.0,0.000,0.0,0.792,38.0,1.389,74.0,141.0,1.155,...,0.0,66.0,0.000,-0.078,-1,66.0,NaN,0,169.0,0.000
3,0.196,74.0,0.000,0.0,0.947,56.0,0.000,0.0,439.0,-0.339,...,0.0,0.0,0.000,0.000,-1,0.0,114.0,1,79.0,0.000
4,0.287,102.0,0.000,0.0,1.021,36.0,0.000,0.0,937.0,0.756,...,104.0,66.0,39.000,-0.034,-2,170.0,136.0,1,92.0,6.612


In [4]:
data.columns

Index(['P_Area_I', 'P_PeakTime_I', 'Q_Area_I', 'Q_PeakTime_I', 'R_Area_I',
       'R_PeakTime_I', 'S_Area_I', 'S_PeakTime_I', 'QRS_Balance_I', 'T_Area_I',
       ...
       'T+_Dur_aVF', 'T-_Dur_aVF', 'T+_Amp_aVF', 'T-_Amp_aVF', 'T_Morph_aVF',
       'T_DurFull_aVF', 'P_Dur_Global', 'P_Found_Global', 'HR__Global',
       'P_Term_V1'],
      dtype='object', length=783)

In [5]:
data.shape

(21799, 783)

In [6]:
desc = pd.read_csv(os.path.join("datasets","ptb-xl-a-comprehensive-electrocardiographic-feature-dataset-1.0.1","ptb-xl-a-comprehensive-electrocardiographic-feature-dataset-1.0.1","labels","12sl_statements.csv"))

In [7]:
desc.head()

,ecg_id,statements,statements_cat,statements_ext,statements_ext_snomed
0,1,"['NSR', 'NML']","['NSR', 'NML']","[('NSR', 100.0), ('NML', 100.0)]","[(441840, 100.0), (4008453, 100.0), (4010019, ..."
1,21803,"['SRTH', 'MSAR', 'RBBB', 'IT', 'AB']","['SRTH;MSAR', 'RBBB', 'IT', 'AB']","[('SRTH', 100.0), ('MSAR', 100.0), ('RBBB', 10...","[(134057, 100.0), (313791, 100.0), (314059, 10..."
2,21804,"['AFIB', 'RVR', 'RBBB', 'IT', 'AB']","['AFIB;RVR', 'RBBB', 'IT', 'AB']","[('AFIB', 100.0), ('RVR', 100.0), ('RBBB', 100...","[(134057, 100.0), (313217, 100.0), (313791, 10..."
3,21805,"['NSR', 'LAD3', 'ASMI', 'AU', 'ST&', 'LT', 'AB']","['NSR', 'LAD3', 'ASMI;AU', 'ST&;LT', 'AB']","[('NSR', 100.0), ('LAD3', 100.0), ('ASMI;AU', ...","[(134057, 100.0), (314666, 100.0), (320536, 10..."
4,21806,"['NSR', 'NST', 'AB']","['NSR', 'NST', 'AB']","[('NSR', 100.0), ('NST', 100.0), ('AB', 100.0)]","[(320536, 100.0), (441840, 100.0), (4008453, 1..."


In [8]:
desc.shape

(21799, 5)

In [9]:
desc['statements'].unique()

array(["['NSR', 'NML']", "['SRTH', 'MSAR', 'RBBB', 'IT', 'AB']",
       "['AFIB', 'RVR', 'RBBB', 'IT', 'AB']", ...,
       "['SRTH', 'FREQ', 'PVC', 'ILBBB', 'ST&', 'ALT', 'LNGQT', 'AB']",
       "['ACCEL', 'JUNCT-R', 'OCC', 'PVC', 'ILBBB', 'ST&', 'ALT', 'LNGQT', 'AB']",
       "['STACH', 'OCC', 'PVC', 'ST&', 'LT', 'AB']"], dtype=object)

In [10]:
desc['statements']

0                                          ['NSR', 'NML']
1                    ['SRTH', 'MSAR', 'RBBB', 'IT', 'AB']
2                     ['AFIB', 'RVR', 'RBBB', 'IT', 'AB']
3        ['NSR', 'LAD3', 'ASMI', 'AU', 'ST&', 'LT', 'AB']
4                                    ['NSR', 'NST', 'AB']
                               ...                       
21794                                      ['NSR', 'NML']
21795                               ['SBRAD', 'AT', 'AB']
21796                   ['SRTH', '$SWITH', 'PSVC', 'ABR']
21797                                    ['SBRAD', 'ABR']
21798                             ['SBRAD', 'RBBB', 'AB']
Name: statements, Length: 21799, dtype: object

In [11]:
diseases = [ast.literal_eval(s) for s in desc['statements']]
diseases

[['NSR', 'NML'],
 ['SRTH', 'MSAR', 'RBBB', 'IT', 'AB'],
 ['AFIB', 'RVR', 'RBBB', 'IT', 'AB'],
 ['NSR', 'LAD3', 'ASMI', 'AU', 'ST&', 'LT', 'AB'],
 ['NSR', 'NST', 'AB'],
 ['NSR', 'NML'],
 ['NSR', 'NML'],
 ['NSR', 'IMI', 'AU', 'AB'],
 ['NSR', 'NML'],
 ['NSR', 'NML'],
 ['SRTH', 'FAV', 'RBBB', 'AFB', 'BIFB', 'AB'],
 ['SBRAD', 'QRSV', 'LPAREN', 'SOKOLYON', 'RPAREN', 'BORDE'],
 ['NSR', 'NML'],
 ['SRTH',
  '$SWITH',
  'PAC',
  '$SWITH',
  '$SABCOND',
  'RAD',
  'QRSV',
  'LPAREN',
  'CORNPROD',
  'RPAREN',
  'BORDE'],
 ['NSR', 'NML'],
 ['NSR', 'NML'],
 ['NSR', 'LVH3', 'LPAREN', 'RAVL', 'COMMA', 'CORNPROD', 'RPAREN', 'NST', 'AB'],
 ['NSR', 'QRSV', 'LPAREN', 'CORNPROD', 'RPAREN', 'IMI', 'AU', 'AB'],
 ['NSR', 'NML'],
 ['NSR', 'NML'],
 ['SRTH', 'SAR', 'FAV', 'AFB', 'IMI', 'AU', 'CRO', 'ASMI', 'AU', 'AB'],
 ['NSR', 'NML'],
 ['NSR', 'SAR', 'NML'],
 ['NSR', 'SAR', 'NML'],
 ['SBRAD', 'SPR', 'ABR'],
 ['NSR', 'NML'],
 ['NSR', 'SAR', 'NML'],
 ['NSR', 'SAR', 'NML'],
 ['NSR', 'RAD', 'BORDE'],
 ['FLUT', 'VA

In [12]:
#finding the unique diseases from the target list
from pprint import pprint
unique_values = set()
for el in diseases:
    unique_values.update(el)
pprint ("total unique values: {} {}".format(len(unique_values) ,unique_values))


("total unique values: 167 {'LIOHAI', 'EAR', 'SVR', 'CJP', 'AINJ', 'EATACH', "
 "'ALT', 'W3T1', 'CHB', 'LNGQT', 'NML', 'IIOHAI', 'STACH', 'JESC', 'MIT', "
 "'NT', 'LMI', 'AB', 'RAE', 'ASVPR', 'MILT', 'FAV', 'LVH3', 'SCX', 'ALIHAI', "
 "'MSBRAD', 'SMI', 'PCARD', 'FREQ', 'VPCX', 'BIGEM', '$SIVR', 'LPAREN', "
 "'AVDIS', 'RVH', 'SBRAD', 'APCX', 'RBBB', 'WQTACH', 'ILIHAI', 'SRTH', "
 "'SERYR1', '$SRETC', 'JTACH', 'LVH2', 'RVE+', 'PVC', 'PO', 'JSTN', 'RAD5', "
 "'RVR', 'ILT', 'PAC', 'AIOHAI', 'IRBBB', 'STEMI', 'W4T1', 'FLUT', 'QESPMI', "
 "'UR', 'SVT', 'WPWB', 'QRST', 'JUNBRAD', 'LSBINJ', 'VAVB', 'SAR', 'MSAR', "
 "'EABRAD', 'SVCX', 'MBZI', 'PFB', 'RSR', 'REPOL', 'PROAV', 'ROMESTES', "
 "'$SWITH', 'COMMA', 'IVCD', 'CRO', 'SAV', 'RPAREN', 'INDAX', 'LT', 'AT', "
 "'W2T1', 'CRVI', 'SOKOLYON', '2ST', 'VPR', 'BIFB', 'IVCB', 'BAE', 'ST&', "
 "'QRSV', '$SABCOND', 'MSTDAL', 'VESC', 'ASMI', 'WPWA', 'AVDPR', 'NSR', 'AFB', "
 "'RAVL', 'IINJ', 'POSTMI', 'JUNCT-R', '$SFUS', 'BORDE', 'QCERR', 'SPR', "
 "'

In [13]:
main_classes = ['NORM','STTC','MI','CD','HYP']   
target_classes = ['IPMI','SMI','IT','NSTT','JST','POSTMI','IVCD','STEMI','QRST','IMI','MIT','MILT']

In [14]:
print(len(main_classes))
print(len(data))
SIZE = len(data)

5
21799


In [15]:
def makeTargetLabelIndex(diseases,required,SIZE):
    ans = []
    for i in range(0,SIZE):
        for elm in diseases[i]:
            if elm in required:
                ans.append(i)
    return ans

In [16]:
target_index= makeTargetLabelIndex(diseases=diseases, required=target_classes ,SIZE=SIZE)
print(f'There are total {len(target_index)} heart disease patients and {SIZE - len(target_index)} healthy patients')

There are total 4751 heart disease patients and 17048 healthy patients


In [17]:
def CreateTargetLabels(indexes,size):
    labels = [0]*size
    for el in indexes:
        labels[el] = 1
    return labels
    

In [18]:
labels = CreateTargetLabels(target_index,SIZE)

In [19]:
df = data.copy()
df['target'] = labels

In [20]:
df.head()

,P_Area_I,P_PeakTime_I,Q_Area_I,Q_PeakTime_I,R_Area_I,R_PeakTime_I,S_Area_I,S_PeakTime_I,QRS_Balance_I,T_Area_I,...,T-_Dur_aVF,T+_Amp_aVF,T-_Amp_aVF,T_Morph_aVF,T_DurFull_aVF,P_Dur_Global,P_Found_Global,HR__Global,P_Term_V1,target
0,0.264,64.0,0.000,0.0,0.737,30.0,0.000,0.0,629.0,0.819,...,0.0,0.151,0.000,1,208.0,112.0,1,64.0,0.000,0
1,-0.047,6.0,0.015,8.0,0.794,36.0,1.246,76.0,205.0,2.206,...,127.0,0.000,-0.078,-1,127.0,66.0,1,75.0,0.000,1
2,0.000,0.0,0.000,0.0,0.792,38.0,1.389,74.0,141.0,1.155,...,66.0,0.000,-0.078,-1,66.0,NaN,0,169.0,0.000,1
3,0.196,74.0,0.000,0.0,0.947,56.0,0.000,0.0,439.0,-0.339,...,0.0,0.000,0.000,-1,0.0,114.0,1,79.0,0.000,0
4,0.287,102.0,0.000,0.0,1.021,36.0,0.000,0.0,937.0,0.756,...,66.0,39.000,-0.034,-2,170.0,136.0,1,92.0,6.612,0


In [21]:
X = df.drop('target',axis=1)
y = df['target']

In [22]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X_norm,y,test_size=0.2,random_state=42)

In [24]:
model = Sequential([
    Dense(783, activation='relu', input_shape=(x_train.shape[1],)),
    Dropout(0.5),  # Adding dropout to prevent overfitting
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary crossentropy loss for binary classification
              metrics=['accuracy'])
model.summary()

#Train the model
#history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
#test_loss, test_acc = model.evaluate(x_test, y_test)
#print('Test accuracy:', test_acc)




C:\Users\harsh\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 783)                 │         613,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 783)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │         100,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 722,545 (2.76 MB)

 Trainable params: 722,545 (2.76 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define the input shape
input_shape = (783, 1)

# Create the model
model = Sequential()

# Add three 1D convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add a dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
# Add the output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


C:\Users\harsh\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 781, 32)             │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 390, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 388, 64)             │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 194, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 192, 128)            │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 96, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12288)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │         786,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 825,985 (3.15 MB)

 Trainable params: 825,985 (3.15 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
hist = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
436/436 ━━━━━━━━━━━━━━━━━━━━ 30s 45ms/step - accuracy: 0.7815 - loss: 0.5724 - val_accuracy: 0.7910 - val_loss: 0.5131
Epoch 2/20
436/436 ━━━━━━━━━━━━━━━━━━━━ 33s 75ms/step - accuracy: 0.7882 - loss: 0.5169 - val_accuracy: 0.7910 - val_loss: 0.5129
Epoch 3/20
 16/436 ━━━━━━━━━━━━━━━━━━━━ 38s 91ms/step - accuracy: 0.7990 - loss: 0.5028

KeyboardInterrupt: 

In [27]:
feature_model = Model(inputs=model.inputs, outputs = model.layers[-2].output)
feature_model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 783, 1)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 781, 32)             │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 390, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 388, 64)             │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 194, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 192, 128)            │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 96, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12288)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │         786,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │           8,320 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 825,856 (3.15 MB)

 Trainable params: 825,856 (3.15 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
feature_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
features = feature_model.predict(x_train,verbose=True)


545/545 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step


In [29]:
print(features[0])

[0.08756138 0.10286807 0.08926049 0.         0.         0.10615167
 0.         0.1026435  0.         0.0966289  0.08708207 0.10010469
 0.08974194 0.         0.         0.08714856 0.         0.
 0.09145278 0.         0.09546466 0.         0.         0.
 0.         0.10705742 0.         0.         0.0893026  0.
 0.0980067  0.08686609 0.10329755 0.         0.09038778 0.09776095
 0.         0.09308639 0.12408512 0.08958425 0.         0.
 0.         0.         0.         0.08697341 0.10383845 0.
 0.         0.         0.08690299 0.         0.1053621  0.12712608
 0.         0.         0.08800992 0.09341423 0.09240063 0.
 0.08842342 0.09201603 0.09694425 0.08891505 0.09029803 0.09328535
 0.08968493 0.         0.         0.0868434  0.0877047  0.
 0.11393858 0.09185364 0.10122766 0.09542827 0.         0.11065021
 0.         0.         0.         0.         0.09025146 0.
 0.         0.08773212 0.08973876 0.         0.0891455  0.08922926
 0.         0.         0.09195707 0.11168338 0.08881284 0.


In [30]:

# Evaluate the model on the test set
loss, acc = model.evaluate(x_test, y_test)
print('Test accuracy:', acc)


137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.7788 - loss: 0.5285
Test accuracy: 0.7940366864204407


In [37]:
from keras.models import load_model
keras.saving.save_model(feature_model,filepath="C:\\Users\\harsh\\ML\\Research-Project\\ecg_feature_model.keras")


In [38]:
new = load_model('ecg_feature_model.keras')

ValueError: Layer node index out of bounds.
inbound_layer = <Conv1D name=conv1d, built=True>
inbound_layer._inbound_nodes = [<Node operation=<Conv1D name=conv1d, built=True>, id=1687979950992>]
inbound_node_index = 9